### In-Game Purchase Data Analysis

#### Background 

In this homework, you are ask to deal with a task of analyzing an "in-game purchase" dataset. In this game, players are able to to purchase optional items that enhance their playing experience. Now your task is to generate a report that breaks down the game's purchasing data into meaningful insights.

-----

#### Observable Trends

* There are 1163 active players. The vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.79%) with secondary groups falling between 15-19 (18.58%) and 25-29 (13.37%). 

* The age group that spends the most money is the 20-24 with 1,114.06 dollars as total purchase value and an average purchase of 4.32. In contrast, the demographic group that has the highest average purchase is the 35-39 with 4.76 and a total purchase value of 147.67. 
-----


In [1]:
import pandas as pd
import numpy as np

# Raw data file
file_to_load = "purchase_data.csv"

# Read purchasing file and store into pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [2]:
player_count = len(purchase_data.drop_duplicates(subset="SN"))
pd.DataFrame({"Total Players": [player_count]})

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.
* Create a summary data frame to hold the results
* Display the summary data frame

In [3]:
total_revenue = purchase_data["Price"].sum()
purchase_count = len(purchase_data)

total_purchasing_analysis = pd.DataFrame(dict(zip(
    ("Number of Unique Items", "Average Price", "Number of Purchases", "Total Revenue"), 
    [len(purchase_data.drop_duplicates(subset="Item ID")), f"${round(total_revenue / purchase_count, 2)}", purchase_count, f"${round(total_revenue, 2):,.2f}"]
)), index=[0])

total_purchasing_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
pd.DataFrame({
    "Percentage of Players": purchase_data.drop_duplicates(subset="SN")["Gender"].value_counts(normalize=True).mul(100).round(2), 
    "Total Count": purchase_data.drop_duplicates(subset="SN")["Gender"].value_counts()
})

,Percentage of Players,Total Count
Male,84.03,484
Female,14.06,81
Other / Non-Disclosed,1.91,11



## Purchasing Analysis (Gender)

* Run basic calculations to obtain below values by gender
  * Purchase Count
  * Average Purchase Price  
  * Total Purchase Value
  * Average Purchase Total per Person by Gender
* Create a summary data frame to hold the results
* Display the summary data frame

In [5]:
gender_purchasing_analysis = pd.DataFrame({
    "Purchase Count": purchase_data.value_counts(subset="Gender"), 
    "Average Purchase Price": purchase_data.groupby("Gender")["Price"].mean().round(5), 
    "Total Purchase Value": purchase_data.groupby("Gender")["Price"].sum(), 
    "Avg Purchase Total per Person": purchase_data.groupby("SN").agg({"Price": sum, "Gender": "first"}).groupby("Gender")["Price"].mean().round(2)
    # "Avg Purchase Total per Person": purchase_data.groupby(["SN", "Gender"])["Price"].sum().groupby("Gender").mean().round(2)
})

gender_purchasing_analysis_display = gender_purchasing_analysis.copy()
gender_purchasing_analysis_display["Avg Purchase Total per Person"] = gender_purchasing_analysis["Avg Purchase Total per Person"].apply(lambda x: f"${x:,.2f}")
gender_purchasing_analysis_display["Average Purchase Price"] = gender_purchasing_analysis["Average Purchase Price"].apply(lambda x: f"{x:g}")
gender_purchasing_analysis_display

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Purchase Total per Person
Gender,,,,
Female,113,3.20301,361.94,$4.47
Male,652,3.01785,1967.64,$4.07
Other / Non-Disclosed,15,3.346,50.19,$4.56


## Age Demographics

* Categorize players using the age groups. Hint: use pd.cut()
  * each group is for 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
* Calculate the numbers and percentages by age group
* Create a summary data frame to hold the results
* round the percentage column to two decimal points
* Display Age Demographics Table

In [6]:
age_demographics = pd.cut(
    purchase_data.drop_duplicates(subset="SN")["Age"], 
    (0, 9, 14, 19, 24, 29, 34, 39, np.inf), right=True, 
    labels=("<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+")
)

pd.DataFrame({
    "Percentage of Players": age_demographics.value_counts(normalize=True).mul(100).round(2), 
    "Total Count": age_demographics.value_counts()
}, index=("<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"))

,Percentage of Players,Total Count
<10,2.95,17
10-14,3.82,22
15-19,18.58,107
20-24,44.79,258
25-29,13.37,77
30-34,9.03,52
35-39,5.38,31
40+,2.08,12


## Purchasing Analysis (Age)

* The below each broken into groups of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Age Group
* Bin the purchase_data data frame by age
* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
* Create a summary data frame to hold the results
* Display the summary data frame

In [7]:
age_category = purchase_data.copy()
age_category["Age"] = pd.cut(
    purchase_data["Age"], 
    (0, 9, 14, 19, 24, 29, 34, 39, np.inf), right=True, 
    labels=("<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+")
)

age_purchasing_analysis = pd.DataFrame({
    "Purchase Count": age_category.value_counts(subset="Age"), 
    "Average Purchase Price": age_category.groupby("Age")["Price"].mean().round(2), 
    "Total Purchase Value": age_category.groupby("Age")["Price"].sum().round(2), 
    "Average Purchase Total per Person": age_category.groupby("SN").agg({"Price": sum, "Age": "first"}).groupby("Age")["Price"].mean().round(2)
}, index=("<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"))

age_purchasing_analysis.agg({
    "Purchase Count": lambda x: x, 
    "Average Purchase Price": lambda x: f"${x:,.2f}", 
    "Total Purchase Value": lambda x: f"${x:,.2f}", 
    "Average Purchase Total per Person": lambda x: f"${x:,.2f}"
})

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Identify the the top 5 spenders in the game by total purchase value (top-5 players with highest total purchase value), then list (in a table):
  * SN(screen name)
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
* Run basic calculations to obtain the results in the table below
* Create a summary data frame to hold the results
* Sort the total purchase value column in descending order
* Display a preview of the summary data frame

In [8]:
top_spenders = pd.DataFrame({
    "Purchase Count": purchase_data.groupby("SN").size(), 
    "Average Purchase Price": purchase_data.groupby("SN")["Price"].mean(), 
    "Total Purchase Value": purchase_data.groupby("SN")["Price"].sum()
})

top_spenders.nlargest(5, columns="Total Purchase Value", keep="all").agg({
    "Purchase Count": lambda x: x, 
    "Average Purchase Price": lambda x: f"${x:,.2f}", 
    "Total Purchase Value": lambda x: f"${x:,.2f}"
})

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.41,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Top 5 most popular items by purchase count:
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value
* Retrieve the Item ID, Item Name, and Item Price columns
* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value
* Create a summary data frame to hold the results
* Sort the purchase count column in descending order
* Display a preview of the summary data frame

In [9]:
items_data = purchase_data.groupby(["Item ID", "Item Name"])
most_popular_items = pd.DataFrame({
    "Purchase Count": items_data.size(), 
    "Item Price": items_data["Price"].first(), 
    "Total Purchase Value": items_data["Price"].sum()
})

most_popular_items.nlargest(5, columns="Purchase Count", keep="all").agg({
    "Purchase Count": lambda x: x, 
    "Item Price": lambda x: f"${x:,.2f}", 
    "Total Purchase Value": lambda x: f"${x:,.2f}"
})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
145,Fiery Glass Crusader,9,$4.58,$41.22
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16
34,Retribution Axe,8,$2.22,$17.76
37,"Shadow Strike, Glory of Ending Hope",8,$3.16,$25.28
59,"Lightning, Etcher of the King",8,$4.23,$33.84
60,Wolf,8,$3.54,$28.32


## Most Profitable Items

* Top 5 most profitable items by total purchase value:
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value
* Sort the above table by total purchase value in descending order
* Display a preview of the data frame

In [10]:
most_popular_items.nlargest(5, columns="Total Purchase Value", keep="all").agg({
    "Purchase Count": lambda x: x, 
    "Item Price": lambda x: f"${x:,.2f}", 
    "Total Purchase Value": lambda x: f"${x:,.2f}"
})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
